# 

# Tutorial 8: Managing Many Tools with Tool Retrieval

As you build more complex agents, the number of available tools can grow quickly. An agent with dozens or even hundreds of tools faces a significant challenge: the more tools it has, the larger the context it must consider, and the higher the chance of it becoming confused or selecting the wrong one.

**Tool Retrieval** is ToolBrain's solution to this problem. It acts as an intelligent filter, dynamically selecting a small, relevant subset of tools based on the user's query, and presenting only those to the agent.

This tutorial is based on the example in `examples/06_tool_retrieval.py`.

## The Problem: Context Overload

When an agent is initialized, the descriptions (docstrings) of all its tools are injected into its system prompt. If you have 50 tools, the system prompt can become enormous. This not only consumes valuable context window space but can also overwhelm the agent, making it harder for it to identify the single best tool for the job.

## The Solution: `ToolRetriever`

The `ToolRetriever` is a component that sits between your full toolset and the agent. Before the agent begins its reasoning process, the retriever steps in.

**How it works:**

1.  You provide the `ToolRetriever` with your full list of tools.
2.  When a query comes in, the `ToolRetriever` uses a separate, external language model (often a fast and cheap one like `gpt-4o-mini`) to read the query and the docstrings of all available tools.
3.  Based on semantic relevance, it selects a small number of tools that are most likely to be useful for solving the query.
4.  Only this filtered subset of tools is passed to the agent for the current task.

This ensures the agent works with a clean, minimal, and highly relevant set of tools for every single query, dramatically improving its focus and performance.

## The Code: `06_tool_retrieval.py`

Let's examine the example script to see how to implement this.

### Step 1: Define a Larger Set of Tools

First, we define several tools. In a real-world scenario, this list could be much longer.

In [1]:
import os
from toolbrain import Brain, create_agent
from toolbrain.rewards import reward_exact_match
from smolagents import tool, CodeAgent, TransformersModel
from toolbrain.retriever import ToolRetriever
from openai import OpenAI

/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/toolbrain/adapters.py:9: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
@tool
def add(a: int, b: int) -> int:
    """
    Add two integers.
    
    Args:
        a: First integer to add
        b: Second integer to add
        
    Returns:
        The sum of a and b
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.
    
    Args:
        a: First integer to multiply
        b: Second integer to multiply
        
    Returns:
        The product of a and b
    """
    return a * b

@tool
def divide(a: int, b: int) -> int:
    """
    Divide two integers.
    
    Args:
        a: Dividend
        b: Divisor
        
    Returns:
        The quotient of a divided by b
    """
    return a / b

@tool
def subtract(a: int, b: int) -> int:
    """
    Subtract two integers.
    
    Args:
        a: First integer
        b: Second integer to subtract from a
        
    Returns:
        The difference of a minus b
    """
    return a - b


# Imagine many more tools here...


### Step 2: Create the code agent

In [3]:
print("📥 Initializing TransformersModel...")
trainable_model = TransformersModel(model_id="Qwen/Qwen2.5-0.5B-Instruct",
                                    max_new_tokens=128)

print("✅ TransformersModel initialized.")

print("🔧 Creating CodeAgent...")
agent = CodeAgent(
    tools=[add, multiply, divide, subtract],
    model=trainable_model,
    max_steps=1
)
print("✅ Agent created.")

📥 Initializing TransformersModel...
✅ TransformersModel initialized.
🔧 Creating CodeAgent...
✅ Agent created.


### Step 3: Initialize the `ToolRetriever`

Next, we create an instance of the `ToolRetriever`. This requires a language model to perform the retrieval task. This can be a different model from the agent's main model.

In [ ]:
retrieval_llm_instance = OpenAI(api_key=os.getenv("OPENAI_API_KEY")).chat.completions.create
retrieval_llm_model = "gpt-4o-mini" 
tool_retriever = ToolRetriever(llm_model=retrieval_llm_model,
                               llm_instance=retrieval_llm_instance,
                               retrieval_topic="mathematics",
                               retrieval_guidelines="Select only tools needed for mathematical calculations")


Here, we've configured the retriever to use `gpt-4o-mini` and given it guidelines to focus its selection on mathematical tools.

### Step 4: Pass the Retriever to the Brain

We create our agent with the *full* list of tools. Then, when we initialize the `Brain`, we simply pass our configured `tool_retriever` instance to it.

In [ ]:
# Agent is aware of all tools initially
# Initialize the training brain  
brain = Brain(
    agent=agent,
    reward_func=reward_exact_match,
    algorithm="GRPO",
    tool_retriever=tool_retriever
)

### Step 5 Prepare the Training dataset

In [ ]:
# Prepare training dataset
training_dataset = [
    {
        "query": "Calculate 5 + 7",
        "gold_answer": "12"
    },
    {
        "query": "What is 8 multiplied by 6?",
        "gold_answer": "48"
    },
    {
        "query": "Calculate 8 divided by 2",
        "gold_answer": "4"
    },
    {
        "query": "Calculate 8 subtracted from 2",
        "gold_answer": "6"
    }
]

### Step 6: Run Training

The rest of the process is exactly the same. When `brain.train()` is called, the `Brain` will automatically use the `tool_retriever` to retrieve relevant tools for each query in a training step.

In [ ]:
brain.train(training_dataset, num_iterations=1)



For a query like `"Calculate 5 + 7"`, the retriever will analyze the query and likely select only the `add` and `subtract` tools as relevant. The agent will then receive a simplified context containing only these two tools, making it much easier for it to choose the correct one.

### Step 7: Retrieve the trained agent to run new queries


In [ ]:
trained_agent = brain.get_agent()

print(trained_agent.run("Calculate 6 + 9"))
print(trained_agent.run("What is 4 multiplied by 2?"))



---

Tool retrieval is an essential feature for building scalable and robust agents. By dynamically managing the complexity of the toolset, you can significantly improve your agent's performance and reliability, especially as your application grows.